# SignSense
Progetto di Sistemi Digitali M - Iacopo Sbalchiero, Pietro Coloretti

## 1. Preparazione dell'ambiente
Importiamo nell'ambiente tutte le librerie necessarie

In [1]:
import torch # YOLOv5 implemented using pytorch
from IPython.display import Image #this is to render predictions

C:\Users\Iacopo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\torch\nn\modules\transformer.py:20: UserWarning: Failed to initialize NumPy: numpy.core.multiarray failed to import (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


Installiamo quindi Yolov5 clonandolo dal repository ufficiale e installiamo tutte le dipendenze richieste:

In [3]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...


Importiamo gli ultimi pacchetti per analizzare lo stato dell'allenamento:

In [9]:
from tqdm import tqdm
import IProgress
from ipywidgets import IntProgress

In [11]:
print('All set. Using PyTorch version %s with %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

All set. Using PyTorch version 2.2.1+cpu with CPU


## 2. Preparazione del dataset

Scarichiamo il dataset da Roboflow

In [12]:
!pip install --user roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="####")
project = rf.workspace("signsense-0gsgr").project("signsense-gnn0l")
version = project.version(1)
dataset = version.download("yolov5")

You should consider upgrading via the 'C:\Users\Iacopo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


  Attempting uninstall: idna
    Found existing installation: idna 3.6
    Uninstalling idna-3.6:
      Successfully uninstalled idna-3.6
  Attempting uninstall: certifi
    Found existing installation: certifi 2024.2.2
    Uninstalling certifi-2024.2.2:
      Successfully uninstalled certifi-2024.2.2
  Attempting uninstall: cycler
    Found existing installation: cycler 0.12.1
    Uninstalling cycler-0.12.1:
      Successfully uninstalled cycler-0.12.1
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to SignSense-1 in yolov5pytorch:: 100%|██████████| 1656/1656 [00:00<00:00, 1754.28it/s]


Spostiamo il dataset in ../dataset e creiamo il file dataset.yaml come segue:
```
train: ../dataset/train/images
val: ../dataset/valid/images

no: 29
names: ['a', 'b', 'c', 'd', 'del', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 'reset', 's', 'sapce', 't', 'u', 'v', 'w', 'x', 'y', 'z']
```

## 3. Training
Eseguiamo il train. Il flag ```--img``` indica la dimensione delle immagini usate per il dataset.

In [14]:
!python train.py --img 640 --batch 16 --epochs 50 --data dataset.yaml --weights yolov5s.pt --cache

Alla fine dell'allenamento, due file dovrebbero essere salvati in ```yolov5/runs/train/exp/weights```: ```last.pt``` e ```best.pt```. Usiamo ```best.pt``` contenente il modello con le statistiche migliore.

Possiamo tracciare un batch di validazione ottenuto durante l'addestramento e controllare il punteggio di confidenza di ciascuna etichetta:

In [ ]:
Image(filename='runs/train/exp2/val_batch0_labels.jpg', width=1000)

## 4. Testing del modello

In [ ]:
!python detect.py --weights runs/train/exp2/weights/best.pt --img 416 --conf 0.25 --source runs/train/exp2/test_images

## 5. Esportazione nel formato .tflite

In [ ]:
# Convert weights to fp16 TFLite model

!python export.py --weights runs/train/exp2/weights/best.pt --include tflite --img 640

Verifichiamo il modello ```.tflite``` appena creato:

In [ ]:
#After running the above command, you will have best-fp16.tflite in runs\train\exp2\weights folder

# Lets run the created tflite model.

!python detect.py --weights runs/train/exp2/weights/best-fp16.tflite --img 640 --conf 0.25 --source runs/train/exp2/test_images

Possiamo anche ottenere il modello quantizzato con il flag ```--int8```:

In [ ]:
#Convert weights to int8 TFLite model

#!python export.py --weights runs/train/exp2/weights/best.pt --include tflite --int8 --img 640 --data data/coco128.yaml